<a href="https://colab.research.google.com/github/hideousmaiden/youngtyp_corpora/blob/sasha/youngtyp_corpora_parsing%2Bsql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
import re
import sqlite3
from tqdm.auto import tqdm
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
conn = sqlite3.connect('youngtyp.db')
cur = conn.cursor()

In [ ]:
# создаю пустую базу

cur.execute("""
CREATE TABLE IF NOT EXISTS texts
(id_text int, text text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS meta
(id_text int, author text, topic text, year int, affilation text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS sents 
(id_text int, id_sent int, sent text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS words
(id_text int, id_sent int, id_word int, word text collate nocase, lemma text, pos text)
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS words_right
(id_text int, id_sent int, id_word_right int, word_right text collate nocase, lemma_right text, pos_right text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS words_rright
(id_text int, id_sent int, id_word_rright int, word_rright text collate nocase, lemma_rright text, pos_rright text)
""")


conn.commit()  # создаю базу

In [ ]:
!wget -r 'https://raw.githubusercontent.com/hideousmaiden/youngtyp_corpora/sasha/theses.json'

--2022-10-24 09:48:10--  https://raw.githubusercontent.com/hideousmaiden/youngtyp_corpora/sasha/theses.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3733358 (3.6M) [text/plain]
Saving to: ‘raw.githubusercontent.com/hideousmaiden/youngtyp_corpora/sasha/theses.json’

raw.githubuserconte 100%[===================>]   3.56M  --.-KB/s    in 0.08s   

2022-10-24 09:48:10 (42.6 MB/s) - ‘raw.githubusercontent.com/hideousmaiden/youngtyp_corpora/sasha/theses.json’ saved [3733358/3733358]

FINISHED --2022-10-24 09:48:10--
Total wall clock time: 0.6s
Downloaded: 1 files, 3.6M in 0.08s (42.6 MB/s)


In [ ]:
with open('raw.githubusercontent.com/hideousmaiden/youngtyp_corpora/sasha/theses.json', 'r', encoding="utf-8") as f:
  bodies = json.load(f)

In [ ]:
# writing down raw text and sentences
for id, t_id in enumerate(bodies.keys()):
    cur.execute("""INSERT INTO texts
                (id_text, text) 
                VALUES (?, ?)""", (id, bodies[t_id]['text']))
    cur.execute("""INSERT INTO meta
                (id_text, author, topic, year, affilation) 
                VALUES (?, ?, ?, ?, ?)""", (id,
                                         bodies[t_id]['author'], bodies[t_id]['title'],
                                         bodies[t_id]['year'], bodies[t_id]['affilation']))
    sents = sent_tokenize(bodies[t_id]['text'])
    for spair in enumerate(sents):
      cur.execute("""INSERT INTO sents
                (id_text, id_sent, sent) 
                VALUES (?, ?, ?)""", (id, *spair))
conn.commit()

###  Парсинг

In [ ]:
#!pip install deeppavlov

In [ ]:
import deeppavlov
#!python -m deeppavlov install morpho_ru_syntagrus_pymorphy

In [ ]:
from deeppavlov import build_model, configs

#model = build_model(configs.morpho_tagger.UD2_0.morpho_ru_syntagrus_pymorphy, download=True)

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
# writing down words

cur.execute("""
    SELECT id_text, id_sent, sent
    FROM sents""")
sents = cur.fetchall()
for one in sents:
#    tags = list(filter(lambda x: len(x)>2, tags))
    words = word_tokenize(one[-1])
    words=[word for word in words if word.isalpha()]
    if len(words) == 0:
      continue
    tags = model(words)
    pos = list(map(lambda x: re.split('\t',x)[2] if len(x)>2 else x, tags))
    lems = list(map(lambda x: morph.parse(x)[0].normal_form, words))
    for wrd in zip(range(len(pos)), words, lems, pos):
        cur.execute("""INSERT INTO words
                (id_text, id_sent, id_word, word, lemma, pos) 
                VALUES (?, ?, ?, ?, ?, ?)""", (one[0], one[1], *wrd))
        cur.execute("""INSERT INTO words_right
                (id_text, id_sent, id_word_right, word_right, lemma_right, pos_right) 
                VALUES (?, ?, ?, ?, ?, ?)""", (one[0], one[1], *wrd))
        cur.execute("""INSERT INTO words_rright
                (id_text, id_sent, id_word_rright, word_rright, lemma_rright, pos_rright) 
                VALUES (?, ?, ?, ?, ?, ?)""", (one[0], one[1], *wrd))
conn.commit()

Так как в задании эксплицитно указано, что в поиске будет до трёх слов, поиск нам показалось удобным осуществлять по таблице, где для каждого слова в отдельных колонках указаны его правые соседи и их свойства.



> беда: SQL не может быть case-insensitive для русского

In [6]:
def process_query(query, limit=10):
  """
  process cql query

  args:
    query: str,
  returns:
    result: list of dict, list of dictionaries containing found results
      pattern: str, found pattern
      id_sent: int, 
      id_text: int,
      id_word: int
      (last_word_id: int)
  """
# токены разделены словами
  parts = query.split(' ')
  # условия разделены плюсами
  conds = list(map(lambda x: x.split('+'), parts))
  pos_tr = {0:'pos',1:'pos_right', 2:'pos_rright'}
  lemma_tr = {0:'lemma',1:'lemma_right', 2:'lemma_rright'}
  word_tr = {0:'word',1:'word_right', 2:'word_rright'}
  que = []
  ask = {1:'''SELECT id_text, id_sent, id_word, word FROM big_words
  WHERE {} COLLATE NOCASE
  ORDER BY id_text ASC
  LIMIT {};''', 2:'''SELECT id_text, id_sent, id_word, word, id_word_right, word_right FROM big_words
  WHERE {} COLLATE NOCASE
  ORDER BY id_text ASC
  LIMIT {}
  ;''', 3:'''SELECT id_text, id_sent, id_word, word, id_word_rright, word_rright FROM big_words
  WHERE {} COLLATE NOCASE
  ORDER BY id_text ASC
  LIMIT {}'''}
  # по словам
  for nn, ques in enumerate(conds):
    for cond in ques:
      #все условия для одного слова
      if re.match('[A-Z]+', cond):
          
          que.append('{} = "{}"'.format(pos_tr[nn], cond))
      elif re.match(r'".+"', cond):
          que.append('(({} = {}) '.format(word_tr[nn], cond) + 'OR ({} = {}) '.format(word_tr[nn], cond.lower()) + 'OR ({} = {}))'.format(word_tr[nn], cond.title()))
          #que.append('{} = {}'.format(word_tr[nn], cond))
      else:
          que.append('{} = "{}"'.format(lemma_tr[nn], morph.parse(cond)[0].normal_form))
  big_que = ' AND '.join(que)
  ask = ask[len(conds)].format(big_que, limit)
  cur.execute(ask)
  done = set(cur.fetchall())
  result = []
  for res in list(done):
    dd = dict()
    dd['pattern'] = ' '.join(list(filter(lambda x: str(x).isalpha(), res)))
    dd['id_sent'] = res[1]
    dd['id_text'] = res[0]
    dd['id_word'] = res[2]
    if len(res) > 4:
      dd['last_word_id'] = res[-2]
    result.append(dd)
  return result

In [ ]:
def process_query_old(query, limit=10):
  """
  process cql query

  args:
    query: str,
  returns:
    result: list of dict, list of dictionaries containing found results
      pattern: str, found pattern
      id_sent: int, 
      id_text: int,
      id_word: int
      (last_word_id: int)
  """
# токены разделены словами
  parts = query.split(' ')
  # условия разделены плюсами
  conds = list(map(lambda x: x.split('+'), parts))
  pos_tr = {0:'pos',1:'pos_right', 2:'pos_rright'}
  lemma_tr = {0:'lemma',1:'lemma_right', 2:'lemma_rright'}
  word_tr = {0:'word',1:'word_right', 2:'word_rright'}
  que = []
  ask = {1:'''SELECT words.id_text, words.id_sent, words.id_word, words.word FROM words
  JOIN words_right ON words_right.id_sent=words.id_sent AND words_right.id_text=words.id_text AND id_word_right = (id_word + 1)
  JOIN words_rright ON words_rright.id_sent=words.id_sent AND words_rright.id_text=words.id_text AND id_word_rright = (id_word + 2)
  WHERE {} COLLATE NOCASE
  LIMIT {};''', 2:'''SELECT words.id_text, words.id_sent, words.id_word, words.word, id_word_right, word_right FROM words
  JOIN words_right ON words_right.id_sent=words.id_sent AND words_right.id_text=words.id_text AND id_word_right = (id_word + 1)
  JOIN words_rright ON words_rright.id_sent=words.id_sent AND words_rright.id_text=words.id_text AND id_word_rright = (id_word + 2)
  WHERE {} COLLATE NOCASE
  LIMIT {}
  ;''', 3:'''SELECT words.id_text, words.id_sent, words.id_word, words.word, id_word_rright, word_rright FROM words
  JOIN words_right ON words_right.id_sent=words.id_sent AND words_right.id_text=words.id_text AND id_word_right = (id_word + 1)
  JOIN words_rright ON words_rright.id_sent=words.id_sent AND words_rright.id_text=words.id_text AND id_word_rright = (id_word + 2)
  WHERE {} COLLATE NOCASE
  LIMIT {}'''}
  # по словам
  for nn, ques in enumerate(conds):
    for cond in ques:
      #все условия для одного слова
      if re.match('[A-Z]+', cond):
          
          que.append('{} = "{}"'.format(pos_tr[nn], cond))
      elif re.match(r'".+"', cond):
          que.append('(({} = {}) '.format(word_tr[nn], cond) + 'OR ({} = {}) '.format(word_tr[nn], cond.lower()) + 'OR ({} = {}))'.format(word_tr[nn], cond.title()))
          #que.append('{} = {}'.format(word_tr[nn], cond))
      else:
          que.append('{} = "{}"'.format(lemma_tr[nn], morph.parse(cond)[0].normal_form))
  big_que = ' AND '.join(que)
  ask = ask[len(conds)].format(big_que, limit)
  cur.execute(ask)
  done = set(cur.fetchall())
  result = []
  for res in list(done):
    dd = dict()
    dd['pattern'] = ' '.join(list(filter(lambda x: str(x).isalpha(), res)))
    dd['id_sent'] = res[1]
    dd['id_text'] = res[0]
    dd['id_word'] = res[2]
    if len(res) > 4:
      dd['last_word_id'] = res[-2]
    result.append(dd)
  return result

In [7]:
process_query("ADP NOUN")

[{'pattern': 'в работе',
  'id_sent': 9,
  'id_text': 0,
  'id_word': 1,
  'last_word_id': 2},
 {'pattern': 'в зависимости',
  'id_sent': 1,
  'id_text': 0,
  'id_word': 9,
  'last_word_id': 10},
 {'pattern': 'с использованием',
  'id_sent': 2,
  'id_text': 0,
  'id_word': 8,
  'last_word_id': 9},
 {'pattern': 'с адресатом',
  'id_sent': 2,
  'id_text': 0,
  'id_word': 15,
  'last_word_id': 16},
 {'pattern': 'от пола',
  'id_sent': 1,
  'id_text': 0,
  'id_word': 11,
  'last_word_id': 12},
 {'pattern': 'с неаргументом',
  'id_sent': 0,
  'id_text': 0,
  'id_word': 12,
  'last_word_id': 13}]

In [ ]:
cur.execute("""
CREATE TABLE IF NOT EXISTS big_words
(id_text int, id_sent int, id_word int, word text, lemma text, pos text, id_word_right int, word_right text,
lemma_right text, pos_right text, id_word_rright int, word_rright text, lemma_rright text, pos_rright text)
""")


conn.commit()

In [ ]:
q = '''SELECT * FROM words
  JOIN words_right ON words_right.id_sent=words.id_sent AND words_right.id_text=words.id_text AND id_word_right = (id_word + 1)
  JOIN words_rright ON words_rright.id_sent=words.id_sent AND words_rright.id_text=words.id_text AND id_word_rright = (id_word + 2)'''
cur.execute(q)
ll = cur.fetchall()

In [ ]:
ll[0]

(0,
 0,
 0,
 'Настоящий',
 'настоящий',
 'ADJ',
 0,
 0,
 1,
 'доклад',
 'доклад',
 'NOUN',
 0,
 0,
 2,
 'посвящен',
 'посвятить',
 'VERB')

In [ ]:
for line in ll:
  cur.execute("""INSERT INTO big_words
                (id_text, id_sent, id_word, word, lemma, pos, 
                id_word_right, word_right, lemma_right, pos_right,
                id_word_rright, word_rright, lemma_rright, pos_rright) 
                VALUES (?, ?, ?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?, ?)""",
              (*line[:6], *line[8:12], *line[14:]))
conn.commit()

In [ ]:
cur.execute('''DROP TABLE words;''')
cur.execute('''DROP TABLE words_right;''')
cur.execute('''DROP TABLE words_rright;''')

In [ ]:
conn.commit()

In [ ]:
conn.close()

In [3]:
conn = sqlite3.connect('youngtyp_smol.db')
cur = conn.cursor()

In [ ]:
cur.execute("""
CREATE TABLE IF NOT EXISTS texts
(id_text int, text text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS meta
(id_text int, author text, topic text, year int, affilation text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS sents 
(id_text int, id_sent int, sent text) 
""")
cur.execute("""
CREATE TABLE IF NOT EXISTS big_words
(id_text int, id_sent int, id_word int, word text, lemma text, pos text, id_word_right int, word_right text,
lemma_right text, pos_right text, id_word_rright int, word_rright text, lemma_rright text, pos_rright text)
""")


conn.commit()

In [ ]:
cur.execute("""SELECT * FROM big_words
LIMIT 105""")
wrds = cur.fetchall()
cur.execute("""SELECT * FROM texts
LIMIT 105""")
thts = cur.fetchall()
cur.execute("""SELECT * FROM sents LIMIT 105""")
snts = cur.fetchall()
cur.execute("""SELECT * FROM meta
LIMIT 105""")
mts = cur.fetchall()

In [ ]:
len(wrds)

105

In [ ]:
for kk in wrds:
    cur.execute("""INSERT INTO big_words (id_text, id_sent, id_word, word, lemma, pos, 
                id_word_right, word_right, lemma_right, pos_right,
                id_word_rright, word_rright, lemma_rright, pos_rright)
                VALUES (?, ?, ?, ?, ?, ?, ?,?, ?, ?, ?, ?, ?, ?)""", kk)
conn.commit()

In [ ]:
for ss in snts:
    cur.execute("""INSERT INTO sents
                (id_text, id_sent, sent) 
                VALUES (?, ?, ?)""", ss)
conn.commit()

In [ ]:
for mm in mts:
    cur.execute("""INSERT INTO meta
                (id_text, author, topic, year, affilation) 
                VALUES (?, ?, ?, ?, ?)""", mm)
conn.commit()

In [ ]:
for tt in thts:
    cur.execute("""INSERT INTO texts
                (id_text, text) 
                VALUES (?, ?)""", tt)
conn.commit()

In [ ]:
conn.close()